## TradingEnv-v0

### Open AI 'Gym' for reinforcement-learning based trading algorithms

This gym implements a very simple trading environment for reinforcement learning.

The gym provides daily observations based on real market data pulled from Quandl on, by default, the SPY etf.  An episode is defined as 252 contiguous days sampled from the overall dataset.  Each day is one 'step' within the gym and for each step, the algo has a choice:

 - SHORT (0)
 - FLAT (1)
 - LONG  (2)
 
If you trade, you will be charged, by default, 10 BPS of the size of your trade.  Thus, going from short to long costs twice as much as going from short to/from flat.  Not trading also has a default cost of 1 BPS per step.  Nobody said it would be easy!
 
At the beginning of your episode, you are allocated 1 unit of cash.  This is your starting Net Asset Value (NAV). 

### Beating the trading game 

For our purposes, we'll say that beating a buy & hold strategy, on average, over one hundred episodes will notch a win to the proud ai player.  We'll illustrate exactly what that means below.

### Let's look at some code using the environment


###  imports

In [ ]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading

### create the environment

This may take a moment as we are pulling historical data from quandl.

In [ ]:
env = gym.make('trading-v0')
#env.time_cost_bps = 0 #

### the trading model

Each time step is a day.  Each episode is 252 trading days - a year.  Each day, we can choose to be short (0), flat (1) or long (2) the single instrument in our trading universe.

Let's run through a day and stay flat.

In [ ]:
observation = env.reset()
done = False
navs = []
while not done:
    action = 1 # stay flat
    observation, reward, done, info = env.step(action)
    navs.append(info['nav'])
    if done:
        #print 'Annualized return: ',navs[len(navs)-1]-1
        pd.DataFrame(navs).plot()

### Note that you are charged just for playing - to the tune of 1 basis point per day!


### Rendering

For now, no rendering has been implemented for this gym, but with each step, the following datum are provided which you can easily graph and otherwise visualize as we see above with the NAV:

 - pnl - how much did we make or lose between yesterday and today?
 - costs  - how much did we pay in costs today
 - nav    - our current nav
 

## utility methods: running strategies once or repeatedly

Although the gym can be 'exercised' directly as seen above, we've also written utility methods which allow for the running of a strategy once or over many episodes, facilitating training or other sorts of analysis.

To utilize these methods, strategies should be exposed as a function or lambda with the following signature:

`Action a = strategy( observation, environment )`
    
Below, we define some simple strategies and look briefly at their behavior to better understand the trading gym. 

In [ ]:
import trading_env as te

stayflat     = lambda o,e: 1   # stand pat
buyandhold   = lambda o,e: 2   # buy on day #1 and hold
randomtrader = lambda o,e: e.action_space.sample() # retail trader

# to run singly, we call run_strat.  we are returned a dataframe containing 
#  all steps in the sim.
env = env.unwrapped
bhdf = env.run_strat(buyandhold)

print(bhdf.head())

# we can easily plot our nav in time:
bhdf.bod_nav.plot(title='buy & hold nav')

### running the same strategy multiple times will likely yield different results as underlying data changes

In [ ]:
env.run_strat(buyandhold).bod_nav.plot(title='same strat, different results')
env.run_strat(buyandhold).bod_nav.plot()
env.run_strat(buyandhold).bod_nav.plot()

### comparing the buyandhold and random traders

In [ ]:
# running a strategy multiple times should yield insights 
#   into its expected behavior or give it oppty to learn
bhdf = env.run_strats(buyandhold,100)
rdf = env.run_strats(randomtrader,100)

comparo = pd.DataFrame({'buyhold':bhdf.mean(),
                        'random': rdf.mean()})
comparo

## Object of the game

From the above examples, we can see that buying and holding will, over the long run, give you the market return with low costs.

Randomly trading will instead destroy value rather quickly as costs overwhelm.

### So, what does it mean to win the trading game?  

For our purposes, we'll say that beating a buy & hold strategy, on average, over one hundred episodes will notch a win to the proud ai player.

To support this, the trading environment maintains the *mkt_return* which can be compared with the *sim_return*.

Note that the *mkt_return* is frictionless while the *sim_return* incurs both trading costs and the decay cost of 1 basis point per day, so overcoming the hurdle we've set here should be challenging.


### Playing the game: purloined policy gradients

I've taken and adapted (see [code](policy_gradient.py) for details) a policy gradient implementation based on tensorflow to try to play the single-instrument trading game.  Let's see how it does.

In [ ]:
import quandl
import pdb

In [ ]:
Stocks=['A','AAPL']
df = quandl.get_table('WIKI/PRICES', ticker=Stocks, qopts = { 'columns': ['ticker', 'volume','adj_close'] }, date = { 'gte': '2011-12-31', 'lte': '2016-12-31' }, paginate=False) 

df = df[ ~np.isnan(df.volume)][['ticker','volume', 'adj_close']]
#print(df.shape)
# we calculate returns and percentiles, then kill nans
df = df[['ticker','adj_close','volume']] 
df.volume.replace(0,1,inplace=True) # days shouldn't have zero volume..
df['Return'] = (df.adj_close-df.adj_close.shift())/df.adj_close.shift()
#df['Return2Day'] = (df.adj_close-df.adj_close.shift(periods=2))/df.adj_close.shift(periods=2)
#df['Return5Day'] = (df.adj_close-df.adj_close.shift(periods=5))/df.adj_close.shift(periods=5)
#df['Return10Day'] = (df.adj_close-df.adj_close.shift(periods=10))/df.adj_close.shift(periods=10)
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
names=["Stock"+str(i) for i in range(1,3)]

In [ ]:
for i ,j in enumerate(Stocks):
    
    print(i)
    if i==0:
        DF=df[df['ticker'] == Stocks[i]].drop("ticker", axis=1 )
    elif i==1:
        stock1=df[df['ticker'] == Stocks[i]].drop("ticker", axis=1 )
        stock1=  stock1.set_index(np.arange(0,len(stock1)))
        DF=DF.join(stock1, lsuffix='Stock1', rsuffix='Stock2')
       
    else:
        stock1=df[df['ticker'] == Stocks[i]].drop("ticker", axis=1 )
        stock1=  stock1.set_index(np.arange(0,len(stock1)))
        DF.join(df[df['ticker'] == Stocks[i]].drop("ticker", axis=1 ), rsuffix=names[i-2])
        
DF=DF.iloc[10:] # remove 1st 10

In [ ]:
colNames=list(DF);Dim=3
colNames = [i for j, i in enumerate(colNames) if j not in range(Dim-1,2*Dim,Dim)]

DF[colNames] = DF[colNames].apply(lambda x: (x - x.mean()) / (x.var()))

In [ ]:
DF

In [ ]:
df[df['ticker'] == Stocks[i]].drop("ticker", axis=1 )

In [ ]:
df

In [1]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading

env = gym.make('trading-v0')
#env.time_cost_bps = 0 # 

env = env.unwrapped



import tensorflow as tf
import policy_gradient

0
1
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_trading.envs.trading_env.TradingEnv'>' has deprecated methods. Compatibility code invoked.


/Users/andrewplaate/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
INFO:policy_gradient:policy_gradient logger started.


In [ ]:
# create the tf session
sess = tf.InteractiveSession()

# create policygradient
pg = policy_gradient.PolicyGradient(sess, obs_dim=6, num_actions=2, learning_rate=1e-2 )

# and now let's train it and evaluate its progress.  NB: this could take some time...
df,sf = pg.train_model(env, episodes=25001, log_freq=100)#, load_model=True)

W1
> /Users/andrewplaate/mlp3/gym_trading/envs/policy_gradient.py(133)tf_policy_forward()
-> logp=h
(Pdb) c
[[-0.49783456 -0.50216544]]
[[-0.49767798 -0.50232208]]
[[-0.49795163 -0.50204837]]
[[-0.49777526 -0.50222468]]
[[-0.49805346 -0.50194651]]
[[-0.49830052 -0.50169957]]
[[-0.49804822 -0.50195181]]
[[-0.49777445 -0.50222552]]
[[-0.49765587 -0.50234419]]
[[-0.49745527 -0.5025447 ]]
[[-0.49803609 -0.50196391]]
[[-0.49798596 -0.50201404]]
[[-0.49812731 -0.50187272]]
[[-0.49804479 -0.50195527]]
[[-0.49801302 -0.50198698]]
[[-0.49801666 -0.50198328]]
[[-0.49787429 -0.50212574]]
[[-0.49826571 -0.50173432]]
[[-0.49815467 -0.5018453 ]]
[[-0.49853173 -0.50146824]]
[[-0.49847671 -0.50152332]]
[[-0.49838755 -0.50161248]]
[[-0.49827924 -0.50172073]]
[[-0.49826413 -0.50173593]]
[[-0.49832997 -0.50167   ]]
[[-0.49831414 -0.50168586]]
[[-0.49727935 -0.50272071]]
[[-0.49783218 -0.50216782]]
[[-0.49775499 -0.50224501]]
[[-0.49798104 -0.50201899]]
[[-0.49784884 -0.50215119]]
[[-0.49786764 -0.5021323

[[ 0.50004447 -0.49995553]]
[[-0.49980691 -0.50019312]]
[[-0.49992141 -0.50007862]]
[[-0.49942181 -0.50057822]]
[[ 0.50025582  0.49974415]]
[[ 0.4997704  -0.50022966]]
[[ 0.4999873 -0.5000127]]
[[ 0.50021237 -0.49978766]]
[[ 0.4998942  -0.50010574]]
[[ 0.50032496 -0.49967507]]
[[ 0.50029653 -0.49970344]]
[[ 0.49978364 -0.50021642]]
[[ 0.49975452 -0.50024545]]
[[ 0.5005855  -0.49941444]]
[[ 0.50028992 -0.49971011]]
[[ 0.49956307 -0.50043684]]
[[ 0.50054139 -0.49945867]]
[[ 0.49987876 -0.50012118]]
[[ 0.50019008 -0.49980995]]
[[ 0.50044918 -0.49955079]]
[[ 0.50035542 -0.49964455]]
[[ 0.50030565 -0.49969432]]
[[ 0.5005402  -0.49945977]]
[[ 0.50070071 -0.4992992 ]]
[[ 0.50032628 -0.49967378]]
[[ 0.50010055 -0.49989948]]
[[ 0.49980086 -0.50019914]]
[[ 0.50048202 -0.49951801]]
[[ 0.50007415 -0.49992585]]
[[ 0.50042701 -0.49957296]]
[[ 0.50040197 -0.49959803]]
[[ 0.50081319 -0.49918678]]
[[ 0.50007457 -0.49992546]]
[[ 0.50024223 -0.49975774]]
[[ 0.50032532 -0.49967474]]
[[ 0.5003252 -0.499674

[[-0.50247675  0.49752331]]
[[-0.50105184  0.49894822]]
[[-0.50079536  0.49920461]]
[[-0.50181788  0.49818212]]
[[-0.50181425  0.49818575]]
[[-0.50201517  0.49798486]]
[[-0.50191927  0.49808076]]
[[-0.50140953  0.49859044]]
[[-0.50200129  0.49799874]]
[[-0.50182778  0.49817225]]
[[-0.50233424  0.49766579]]
[[-0.50204074  0.49795926]]
[[-0.50255632  0.49744365]]
[[-0.50223756  0.49776238]]
[[-0.50249618  0.49750379]]
[[-0.50229698  0.49770302]]
[[-0.50239462  0.49760529]]
[[-0.50241369  0.49758637]]
[[-0.50239158  0.49760848]]
[[-0.5022229   0.49777716]]
[[-0.5022577   0.49774224]]
[[-0.50206417  0.49793583]]
[[-0.50255567  0.49744436]]
[[-0.50297338  0.49702665]]
[[-0.50250489  0.49749509]]
[[-0.50268328  0.49731672]]
[[-0.50248289  0.49751717]]
[[-0.50209671  0.49790326]]
[[-0.502303    0.49769694]]
[[-0.50215685  0.49784309]]
[[-0.50265545  0.49734449]]
[[-0.50220156  0.49779841]]
[[-0.50237095  0.49762911]]
[[-0.50231045  0.49768949]]
[[-0.50262314  0.49737689]]
[[-0.5024851   0.497

[[ 0.49864605 -0.50135392]]
[[ 0.49829566 -0.50170428]]
[[ 0.49860027 -0.50139976]]
[[ 0.49868813 -0.5013119 ]]
[[ 0.49719247 -0.5028075 ]]
[[ 0.4976463  -0.50235373]]
[[ 0.49728894 -0.50271106]]
[[ 0.498409   -0.50159103]]
[[ 0.49753886 -0.50246108]]
[[ 0.49828485 -0.50171518]]
[[ 0.4981055  -0.50189453]]
[[ 0.49753338 -0.50246656]]
[[ 0.49789536 -0.50210464]]
[[ 0.49805012 -0.50194991]]
[[ 0.49828947 -0.50171053]]
[[ 0.49800369 -0.50199628]]
[[ 0.49761337 -0.50238657]]
[[ 0.49805072 -0.50194925]]
[[ 0.49769515 -0.50230485]]
[[ 0.49782476 -0.50217521]]
[[ 0.49858779 -0.50141221]]
[[ 0.49814934 -0.50185066]]
[[ 0.49768883 -0.50231111]]
[[ 0.49737471 -0.50262535]]
[[ 0.49757451 -0.50242543]]
[[ 0.49788442 -0.50211567]]
[[ 0.4980706 -0.5019294]]
[[ 0.49818236 -0.50181764]]
[[ 0.49812785 -0.50187212]]
[[ 0.49800247 -0.50199753]]
[[ 0.4983063  -0.50169367]]
[[ 0.49847972 -0.50152022]]
[[ 0.49842435 -0.50157571]]
[[ 0.49744523 -0.50255477]]
[[ 0.49856186 -0.50143814]]
[[ 0.49858311 -0.50141

[[ 0.49603796 -0.50396204]]
[[-0.50041997 -0.49958006]]
[[-0.50135785 -0.49864215]]
[[-0.50092006 -0.49907988]]
[[-0.50121987  0.49878016]]
[[-0.50005865  0.49994138]]
[[-0.50060523 -0.49939477]]
[[-0.5006597   0.49934033]]
[[-0.49691468 -0.50308532]]
[[-0.50105327  0.49894676]]
[[-0.50111026  0.49888971]]
[[-0.4979226  -0.50207734]]
[[-0.50101566 -0.49898434]]
[[-0.50003821 -0.49996179]]
[[-0.50097287  0.49902716]]
[[-0.50151718 -0.49848285]]
[[-0.50091225  0.49908781]]
[[-0.5013364   0.49866363]]
[[-0.5014993  0.4985007]]
[[-0.49905184 -0.50094813]]
[[-0.50191456 -0.49808547]]
[[-0.50132912  0.49867091]]
[[-0.50053096 -0.49946895]]
[[-0.50188631  0.49811363]]
[[-0.50064641  0.49935359]]
[[-0.49864566 -0.50135428]]
[[-0.50053018  0.49946973]]
[[-0.5017938   0.49820629]]
[[-0.50158733  0.49841261]]
[[-0.50013864 -0.49986142]]
[[-0.4991053  -0.50089473]]
[[-0.50239843  0.49760154]]
[[-0.50193304  0.4980669 ]]
[[-0.50232267  0.49767733]]
[[-0.50237375  0.49762625]]
[[-0.50264919  0.49735

...

[[ 0.50131208 -0.49868801]]
[[ 0.501342   -0.49865794]]
[[ 0.50137711 -0.49862286]]
[[ 0.50116503 -0.49883488]]
[[ 0.50118691 -0.49881306]]
[[ 0.50119233 -0.49880776]]
[[ 0.50141782 -0.49858215]]
[[ 0.50139779 -0.49860215]]
[[ 0.50122488 -0.49877509]]
[[-0.50238049  0.49761948]]
[[-0.49903208  0.50096792]]
[[-0.49931297  0.500687  ]]
[[-0.49909389  0.50090605]]
[[-0.5001592   0.49984086]]
[[-0.4992182   0.50078171]]
[[-0.49921739  0.50078267]]
[[-0.5002259  0.4997741]]
[[-0.50025243  0.49974751]]
[[-0.50011617  0.49988374]]
[[-0.49924445  0.50075555]]
[[-0.50010973  0.49989027]]
[[-0.50005955  0.49994045]]
[[-0.49975085  0.50024921]]
[[-0.50059819  0.49940181]]
[[-0.50057948  0.49942055]]
[[-0.50024503  0.49975497]]
[[-0.49992946  0.50007051]]
[[-0.50019383  0.4998062 ]]
[[-0.49982199  0.50017804]]
[[-0.50038946  0.49961051]]
[[-0.50082338  0.49917665]]
[[-0.50116932  0.49883071]]
[[-0.50069559  0.49930441]]
[[-0.50120479  0.49879527]]
[[-0.50232601  0.49767402]]
[[-0.50104254  0.49895

[[-0.50240678  0.49759313]]
[[-0.50241059  0.49758941]]
[[-0.5024305   0.49756944]]
[[-0.5039615  0.4960385]]
[[-0.50391823  0.49608174]]
[[-0.50284362  0.49715635]]
[[-0.50380921  0.49619082]]
[[-0.50374418  0.49625584]]
[[-0.50245166  0.49754834]]
[[-0.50387216  0.49612784]]
[[-0.50212348  0.4978765 ]]
[[-0.50294191  0.49705815]]
[[-0.50297725  0.49702281]]
[[-0.5036642   0.49633583]]
[[-0.50431192  0.49568814]]
[[-0.50388682  0.49611324]]
[[-0.50311613  0.49688393]]
[[-0.50314844  0.49685165]]
[[-0.50368214  0.49631792]]
[[-0.50403458  0.49596542]]
[[-0.50375968  0.49624038]]
[[-0.50391364  0.49608636]]
[[-0.50366229  0.49633768]]
[[-0.50338125  0.49661881]]
[[-0.5015009  0.4984991]]
[[-0.50191367  0.49808624]]
[[-0.50225556  0.49774444]]
[[-0.50303066  0.49696934]]
[[-0.50359362  0.49640641]]
[[-0.50207257  0.49792752]]
[[-0.5025689  0.4974311]]
[[-0.50275636  0.49724373]]
[[-0.50291455  0.49708536]]
[[-0.50344813  0.4965519 ]]
[[-0.50321758  0.49678236]]
[[-0.50283223  0.49716783]

[[ 0.5021584  -0.49784157]]
[[ 0.50213927 -0.49786076]]
[[ 0.50206304 -0.49793696]]
[[ 0.50212342 -0.49787658]]
[[ 0.50200927 -0.49799073]]
[[ 0.50194758 -0.49805248]]
[[ 0.50207192 -0.49792808]]
[[ 0.50203782 -0.49796215]]
[[ 0.50220942 -0.49779058]]
[[ 0.50201708 -0.49798289]]
[[ 0.50221527 -0.4977847 ]]
[[ 0.50195622 -0.49804372]]
[[ 0.50157446 -0.49842548]]
[[ 0.50167596 -0.49832401]]
[[ 0.50146216 -0.49853793]]
[[ 0.50154459 -0.49845544]]
[[ 0.50197625 -0.49802378]]
[[ 0.50181103 -0.49818903]]
[[ 0.50145251 -0.49854758]]
[[ 0.50162983 -0.4983702 ]]
[[ 0.50172126 -0.4982788 ]]
[[ 0.50152045 -0.49847957]]
[[ 0.50173861 -0.49826145]]
[[ 0.50167203 -0.49832797]]
[[ 0.50157219 -0.49842784]]
[[ 0.50113189 -0.49886814]]
[[ 0.50140923 -0.49859068]]
[[ 0.50119424 -0.49880579]]
[[ 0.50101656 -0.49898347]]
[[ 0.50133997 -0.49866003]]
[[ 0.50122273 -0.49877727]]
[[ 0.50154573 -0.49845427]]
[[ 0.50132781 -0.49867219]]
[[ 0.50135076 -0.49864927]]
[[ 0.50129402 -0.49870604]]
[[ 0.50132519 -0.498

[[-0.50196558  0.49803445]]
[[-0.50202936  0.4979707 ]]
[[-0.50308305  0.49691698]]
[[-0.50355327  0.4964467 ]]
[[-0.50387233  0.49612764]]
[[-0.50326633  0.49673367]]
[[-0.50311285  0.49688712]]
[[-0.50330693  0.4966931 ]]
[[-0.50315619  0.49684379]]
[[-0.50284255  0.49715745]]
[[-0.50345689  0.49654308]]
[[-0.50291741  0.49708259]]
[[-0.50298917  0.49701089]]
[[-0.50358856  0.49641141]]
[[-0.50172293  0.49827701]]
[[-0.5025053   0.49749476]]
[[-0.50329602  0.49670398]]
[[-0.50258464  0.49741533]]
[[-0.5025661   0.49743396]]
[[-0.50412226  0.4958778 ]]
[[-0.50316685  0.49683309]]
[[-0.50351024  0.49648985]]
[[-0.50313783  0.49686211]]
[[-0.50282121  0.49717888]]
[[-0.50377566  0.49622437]]
[[-0.50344235  0.49655759]]
[[-0.50294304  0.49705696]]
[[-0.50299907  0.49700096]]
[[-0.50346231  0.4965376 ]]
[[-0.50292462  0.49707541]]
[[-0.50126982  0.49873015]]
[[-0.50369155  0.49630842]]
[[-0.50258178  0.49741828]]
[[-0.50300598  0.49699393]]
[[-0.50175232  0.49824768]]
[[-0.502509    0.497

[[ 0.49681571 -0.50318426]]
[[ 0.49702123 -0.50297874]]
[[ 0.49661407 -0.5033859 ]]
[[ 0.49731547 -0.50268459]]
[[ 0.49651527 -0.50348479]]
[[ 0.49766058 -0.50233942]]
[[ 0.49762762 -0.50237238]]
[[ 0.49722058 -0.50277948]]
[[ 0.49757746 -0.50242257]]
[[ 0.49779585 -0.50220418]]
[[ 0.49703723 -0.50296283]]
[[ 0.49783641 -0.50216353]]
[[ 0.49819535 -0.50180459]]
[[-0.49749985 -0.50250018]]
[[ 0.49736828 -0.50263172]]
[[ 0.49712908 -0.50287092]]
[[ 0.49723634 -0.50276369]]
[[ 0.49777117 -0.5022288 ]]
[[ 0.49800077 -0.50199926]]
[[ 0.49781165 -0.50218832]]
[[-0.49841085 -0.50158912]]
[[-0.49696976 -0.50303018]]
[[ 0.49709138 -0.50290859]]
[[-0.50365603  0.49634394]]
[[-0.50244117  0.49755886]]
[[-0.50218838  0.49781156]]
[[-0.50446105  0.49553898]]
[[-0.50353408  0.49646586]]
[ 87.14994027]
[[-0.50365698  0.49634302]]
[[-0.50337851  0.4966214 ]]
[[-0.5035454   0.49645457]]
[[-0.50403726  0.4959628 ]]
[[-0.5042668  0.4957332]]
[[-0.50436759  0.4956325 ]]
[[-0.50444895  0.49555105]]
[[-0.50

In [ ]:
while i in range(0,2):
    print(1)

### Results

Policy gradients beat the trading game!  That said, it doesn't work every time and it seems, looking at the charts below, as though it's a bit of a lucky thing.  But luck counts in the trading game as in life!


In [ ]:
sf['net'] = sf.simror - sf.mktror
#sf.net.plot()
sf.net.expanding().mean().plot()
sf.net.rolling(100).mean().plot()

In [ ]:
sf.net.rolling(100).mean().tail()